In [12]:
from google.cloud import storage
import os

# 구글 클라우드 프로젝트 ID
project_id = "clear-talent-315601"
# 구글 클라우드 버킷 이름
bucket_name = "file_bucket_temp"
# 로컬 디렉토리 경로
local_directory_path = "/Users/minjikim/engine-git/crawler-disclosure/files"
# local_directory_path = "/Users/sul-younghwan/crawler/crawler-disclosure/files"
# 인증서 파일 위치 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/minjikim/engine-git/crawler-disclosure/saas-engine-storage.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/sul-younghwan/crawler/crawler-disclosure/saas-engine-storage.json"

## 버킷에 업로드

In [13]:
# 클라이언트 초기화
storage_client = storage.Client(project=project_id)

# 버킷 가져오기
bucket = storage_client.get_bucket(bucket_name)

completed=[]
error_files=[]
# 로컬 디렉토리에서 .pdf 파일 검색 후 버킷에 업로드
pdf_files = [f for f in os.listdir(local_directory_path) if f.endswith("약관.pdf") and '현대해상' in f]

for idx, filename in enumerate(pdf_files):
    file_path = os.path.join(local_directory_path, filename)
    try:
        blob = bucket.blob(filename)
        blob.upload_from_filename(file_path)
        completed.append(idx)
        print(idx,f"{filename} uploaded to {bucket_name}.")
    except:
        error_files.append(idx)
        print("ERROR: ", idx, filename)


['True_현대해상_일반보험_가정종합보험_2011-04-01_2011-05-17_약관.pdf',
 'True_현대해상_일반보험_보통상해보험_2013-04-01_2013-07-22_약관.pdf',
 'True_현대해상_자동차보험_개인용_Hicar다이렉트개인용자동차보험CM_2022-02-01_2022-03-31_약관.pdf',
 'False_현대해상_질병성_무배당모두에게간편한건강보험HI15082종_2015-08-03_2015-12-31_약관.pdf',
 'False_현대해상_종합성_무배당퍼펙트플러스종합보험Hi19012종비갱신납입면제형_2019-01-02_2019-04-01_약관.pdf',
 'False_현대해상_일반보험_국내여행실손의료보험_2018-01-01_2018-03-31_약관.pdf',
 'False_현대해상_종합성_무배당퍼펙트N종합보험Hi1502_2015-04-01_2015-05-31_약관.pdf',
 'False_현대해상_질병성_무배당퍼펙트케어간병보험일반심사Hi2301125종_2023-01-09_2023-03-31_약관.pdf',
 'False_현대해상_상해성_무배당하이콜퍼펙트종합보험연만기갱신형Hi2201_2022-01-01_2022-03-06_약관.pdf',
 'False_현대해상_일반보험_개인실손의료보험_2015-09-01_2015-12-31_약관.pdf',
 'False_현대해상_질병성_무배당하이라이프더블건강암보험Hi22101종3대진단환급형_2022-10-01_2022-12-31_약관.pdf',
 'False_현대해상_상해성_무배당더블간편종합보험Ⅱ세만기형Hi2010간편심사Ⅱ해지환급금미지급형_2020-10-05_2020-12-31_약관.pdf',
 'True_현대해상_일반보험_주택ㆍ온실풍수해보험I_2014-04-01_2014-07-14_약관.pdf',
 'False_현대해상_상해성_무배당하이콜퍼펙트종합보험세만기형Hi22073종해지환급금미지급형_2022-07-01_2022-08-07_약관.pdf',
 'False_현대해상_질병성_무배당암스트롱암보험H

In [3]:
print("총 개수: ",len(pdf_files), "에러 개수: ",len(error_files), "완료된 개수: ", len(completed))

총 개수:  4205 에러 개수:  0 완료된 개수:  4205


## 라벨링 형식 연결, DB CSV 파일 불러오기

In [35]:
import pandas as pd
import re


df= pd.read_csv('aia.csv')

input_string = "~%&*{}\\:<>?/+|\"';,=[]"
output_string = re.sub(r'[~%&*{}\\:<>?/+|\"\';,=\[\]]', '', input_string)

# GCS에 저장된 파일명과 DB에 저장된 파일명이 다르므로 정규표현식으로 처리
df['file'] = df['file'].apply(lambda x: re.sub('[~%&*{}\\:<>?/+|\"\';,=\[\]]', '', x))


len(df)


/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/2407722536.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['file']= df['file'].str.replace('(','')
/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/2407722536.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['file']= df['file'].str.replace(')','')
/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/2407722536.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['file']= df['file'].str.replace('+','')


4230

In [36]:
# GCS 클라이언트 초기화
client = storage.Client()

# 버킷 객체 가져오기
bucket = client.get_bucket(bucket_name)

new_df = pd.DataFrame(columns=["filename"])
# df 데이터프레임 순회하며 public URL 가져오기
for idx, row in df.iterrows():
    # '${보험사}' 텍스트가 포함된 경우에만 처리
    if 'AIA생명' in row['name']:
        # GCS 객체 존재 여부 확인
        blob = bucket.blob(row['file'])
        if blob.exists():
            # public URL 가져오기
            storage_url = blob.public_url
            # URL 업데이트
            df.loc[idx, 'url'] = storage_url
        else:
            new_df = new_df.append({"filename": filename}, ignore_index=True)

/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/382238871.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"filename": filename}, ignore_index=True)
/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/382238871.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"filename": filename}, ignore_index=True)
/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/382238871.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"filename": filename}, ignore_index=True)
/var/folders/07/9kg5067908732vcttk7cnl5m0000gn/T/ipykernel_87228/382238871.py:20: FutureWarning: The frame.append method is deprecated and will be remo

### 체크

In [37]:
error= df[df['url'].str.contains('aia')]
error

,file,name,name.1,start_date,end_date,url
1351,False_AIA생명_방카슈랑스_무배당골든차이나연금보험2011.2.16~2011.7...,AIA생명,무배당골든차이나연금보험(2011.2.16~2011.7.17)/무배당골든차이나연금보험...,2012-04-01,2012-06-30,https://mypage.aia.co.kr/AIAHomepage/disclosur...
1352,False_AIA생명_방카슈랑스_무배당골든차이나연금보험2011.2.16~2011.7...,AIA생명,무배당골든차이나연금보험(2011.2.16~2011.7.17)/무배당골든차이나연금보험...,2012-07-01,2012-08-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
1353,False_AIA생명_방카슈랑스_무배당골든차이나연금보험2011.2.16~2011.7...,AIA생명,무배당골든차이나연금보험(2011.2.16~2011.7.17)/무배당골든차이나연금보험...,2011-07-18,2012-03-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
1354,False_AIA생명_방카슈랑스_무배당골든차이나연금보험2011.2.16~2011.7...,AIA생명,무배당골든차이나연금보험(2011.2.16~2011.7.17)/무배당골든차이나연금보험...,2011-04-01,2011-07-17,https://mypage.aia.co.kr/AIAHomepage/disclosur...
1355,False_AIA생명_방카슈랑스_무배당골든차이나연금보험2011.2.16~2011.7...,AIA생명,무배당골든차이나연금보험(2011.2.16~2011.7.17)/무배당골든차이나연금보험...,2011-02-16,2011-03-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
3475,False_AIA생명_특약_특정신체부위·질병보장제한부인수특약구특정부위·질병부담보특약...,AIA생명,특정신체부위·질병보장제한부인수특약(구:특정부위·질병부담보특약/특별조건부인수특약),2009-06-01,2010-03-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
3476,False_AIA생명_특약_특정신체부위·질병보장제한부인수특약구특정부위·질병부담보특약...,AIA생명,특정신체부위·질병보장제한부인수특약(구:특정부위·질병부담보특약/특별조건부인수특약),2009-04-01,2009-05-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
3477,False_AIA생명_특약_특정신체부위·질병보장제한부인수특약구특정부위·질병부담보특약...,AIA생명,특정신체부위·질병보장제한부인수특약(구:특정부위·질병부담보특약/특별조건부인수특약),2021-01-01,2021-12-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
3478,False_AIA생명_특약_특정신체부위·질병보장제한부인수특약구특정부위·질병부담보특약...,AIA생명,특정신체부위·질병보장제한부인수특약(구:특정부위·질병부담보특약/특별조건부인수특약),2020-04-01,2020-12-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...
3479,False_AIA생명_특약_특정신체부위·질병보장제한부인수특약구특정부위·질병부담보특약...,AIA생명,특정신체부위·질병보장제한부인수특약(구:특정부위·질병부담보특약/특별조건부인수특약),2019-04-01,2020-03-31,https://mypage.aia.co.kr/AIAHomepage/disclosur...


## 버킷 객체 모두 삭제

In [71]:
# # GCS 클라이언트 초기화
# client = storage.Client()

# # 버킷 객체 가져오기
# bucket = client.get_bucket(bucket_name)
# # 
# # new_df = pd.DataFrame(columns=["filename"])
# # fileUrl 데이터프레임 순회하며 public URL 가져오기
# for idx, row in error.iterrows():
#     # '메리츠화재' 텍스트가 포함된 경우에만 처리
#     if '메리츠화재' in row['name']:
#         # GCS 객체 존재 여부 확인
#         blob = bucket.blob(row['url'])
#         if blob.exists():
#             # public URL 가져오기
#             storage_url = blob.public_url
#             # URL 업데이트
#             error.loc[idx, 'url'] = storage_url
#         # else:
#             # new_df = new_df.append({"filename": filename}, ignore_index=True)

## 버킷 특정 텍스트 포함 시 해당 파일 삭제

In [ ]:
# text_to_find = "AIA생명" # 삭제할 파일명에 포함된 텍스트

# # GCS 클라이언트 생성
# client = storage.Client()

# # 버킷 객체 생성
# bucket = client.get_bucket(bucket_name)

# # 버킷 내의 객체 목록 불러오기
# blobs = bucket.list_blobs()

# # 객체 목록에서 특정 텍스트가 포함된 파일 삭제
# for blob in blobs:
#     if text_to_find in blob.name:
#         print(f"Deleting {blob.name}")
#         blob.delete()

In [13]:
error.to_excel('error.xlsx',index=False)

In [1]:
# export file
df.to_excel('aia.xlsx',index=False)

NameError: name 'df' is not defined

## 잘못저장된 값 확인

In [39]:
re=[]
for file_name in os.listdir(local_directory_path):
    if '약관_' in file_name:
        re.append(file_name)
re

[]

## 잘못 저장된 값 삭제

In [24]:
# for file_name in os.listdir(local_directory_path):
#     if '약관_' in file_name:
#         file_path = os.path.join(local_directory_path, file_name)
#         os.remove(file_path)

In [66]:
len(fileUrl[fileUrl['url'].str.contains('www.meritz')])

41

## pdf 파일 개수 확인

In [21]:
tt_files = [f for f in os.listdir(local_directory_path) if f.endswith(".pdf")]

# len(tt_files)
ttt = [f for f in tt_files if f.startswith('True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형')]
ttt.sort()
ttt

['True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2020-11-04_2020-12-31_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-01-01_2021-04-30_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-05-01_2021-05-31_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-05-01_2021-05-31_약관_veLTrOC.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-06-01_2021-07-31_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-06-01_2021-07-31_약관_W4Edaft.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-08-01_2021-11-30_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-08-01_2021-11-30_약관_8jx74pM.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-12-01_2021-12-31_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2021-12-01_2021-12-31_약관_qQuxyLP.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2022-01-01_2022-01-20_약관.pdf',
 'True_AIA생명_건강-상해보험_무배당AIAVitality예방부터케어까지건강보험갱신형_2022-01-01_2022

In [29]:
# for file_name in os.listdir(local_directory_path):
#     if '요약서' in file_name:
#         file_path = os.path.join(local_directory_path, file_name)
#         os.remove(file_path)